In [161]:
import h5py
import xarray as xr
import numpy as np 
from glob import glob 
import geopandas as gpd
from utils.isat2_subs import subset 


In [164]:
# path_isat2 = 'ATL13_20231001151914_01932101_006_02.h5'
paths_isat2 = glob('data/chaohu-lake/icesat2/ATL13/ATL13_*')   ## 1,3
paths_isat2    


['data/chaohu-lake/icesat2/ATL13/ATL13_20210925154349_00491301_006_01.h5',
 'data/chaohu-lake/icesat2/ATL13/ATL13_20210925154349_00491301_007_01.h5']

In [167]:
# path_isat2 = 'data/chaohu-lake/icesat2/ATL13/106770598_ATL13_20240802021955_06972401_006_01_subsetted.h5'
atl13_xr = xr.open_dataset(paths_isat2[0] , group='gt2r', engine='h5netcdf') 
path_lake_vec = 'data/chaohu-lake/hydrolake_chaohu.gpkg'
atl13_xr


<xarray.Dataset> Size: 23MB
Dimensions:                        (delta_time: 57023, ds_sseg_quality: 4)
Coordinates:
  * delta_time                     (delta_time) datetime64[ns] 456kB 2021-09-...
    segment_lat                    (delta_time) float64 456kB ...
    segment_lon                    (delta_time) float64 456kB ...
Dimensions without coordinates: ds_sseg_quality
Data variables: (12/72)
    atl13refid                     (delta_time) float64 456kB ...
    bottom_lat                     (delta_time) float64 456kB ...
    bottom_lon                     (delta_time) float64 456kB ...
    cloud_flag_asr_atl09           (delta_time) float32 228kB ...
    cloud_flag_atm_atl09           (delta_time) float32 228kB ...
    cycle_number                   (delta_time) int8 57kB ...
    ...                             ...
    sseg_start_lon                 (delta_time) float64 456kB ...
    stdev_water_surf               (delta_time) float32 228kB ...
    subsurface_attenuation         (delta_time) float32 228kB ...
    subsurface_backscat_ampltd     (delta_time) float32 228kB ...
    transect_id                    (delta_time) float32 228kB ...
    water_depth                    (delta_time) float32 228kB ...
Attributes:
    Description:         Contains per-beam data products.
    data_rate:           Data within this group are stored at the inland wate...
    atlas_pce:           pce2
    atlas_beam_type:     strong
    groundtrack_id:      gt2r
    atmosphere_profile:  profile_2
    atlas_spot_number:   3
    sc_orientation:      Forward

In [166]:
# path_isat2 = 'data/chaohu-lake/icesat2/ATL13/106770598_ATL13_20240802021955_06972401_006_01_subsetted.h5'
atl13_xr = xr.open_dataset(paths_isat2[1] , group='gt2r', engine='h5netcdf') 
path_lake_vec = 'data/chaohu-lake/hydrolake_chaohu.gpkg'
atl13_xr



<xarray.Dataset> Size: 24MB
Dimensions:                        (delta_time: 56443, ds_sseg_quality: 4)
Coordinates:
  * delta_time                     (delta_time) datetime64[ns] 452kB 2021-09-...
    segment_lat                    (delta_time) float64 452kB ...
    segment_lon                    (delta_time) float64 452kB ...
Dimensions without coordinates: ds_sseg_quality
Data variables: (12/76)
    alpha_mle                      (delta_time) float32 226kB ...
    atl13refid                     (delta_time) float64 452kB ...
    bottom_lat                     (delta_time) float64 452kB ...
    bottom_lon                     (delta_time) float64 452kB ...
    cloud_flag_asr_atl09           (delta_time) float32 226kB ...
    cloud_flag_atm_atl09           (delta_time) float32 226kB ...
    ...                             ...
    sseg_start_lon                 (delta_time) float64 452kB ...
    stdev_water_surf               (delta_time) float32 226kB ...
    subsurface_attenuation         (delta_time) float32 226kB ...
    subsurface_backscat_ampltd     (delta_time) float32 226kB ...
    transect_id                    (delta_time) float32 226kB ...
    water_depth                    (delta_time) float32 226kB ...
Attributes:
    atlas_beam_type:     strong
    atlas_pce:           pce2
    atlas_spot_number:   3
    atmosphere_profile:  profile_2
    data_rate:           Data within this group are stored at the inland wate...
    description:         Contains per-beam data products.
    groundtrack_id:      gt2r
    sc_orientation:      Forward

In [76]:
lake_gdf = gpd.read_file(path_lake_vec)
xmin,ymin,xmax,ymax= lake_gdf.bounds.values[0]
print(xmin, xmax, ymin, ymax)


117.28882025824561 117.85138617621448 31.426250000000266 31.719786512587007


In [77]:
ids = np.where((atl13_xr.segment_lat.values > ymin) & 
               (atl13_xr.segment_lat.values < ymax) &
               (atl13_xr.segment_lon.values > xmin) &
               (atl13_xr.segment_lon.values < xmax))[0]
ids.shape


(313,)

In [78]:
vnames = list(atl13_xr.variables.keys())
vars = [atl13_xr.variables[vname] for vname in vnames] 
vars_dict = [[] for vname in vnames]
vars_dict = dict(zip(vnames, vars))
#### -- subset by specific extent
for vname in vnames: vars_dict[vname] = vars_dict[vname][ids]


In [79]:
# # 保存为 .h5 文件
# with h5py.File('data.h5', 'w') as hf:
#     for key, value in data_dict.items():
#         hf.create_dataset(key, data=value)


In [86]:
# vars_dict['delta_time'].values


In [87]:
# timestamps_str = np.datetime_as_string(vars_dict['delta_time'].values, unit='s')


In [ ]:
#### ----- write out file
ofile = 'tmp_subset.h5'
if vars_dict['segment_lon'].any():
    with h5py.File(ofile, 'w') as fo:
        for vname in vnames:
            if vars_dict[vname].values.dtype == np.dtype('datetime64[ns]'):
                data = vars_dict[vname].values.astype('int64')
                ds = fo.create_dataset(vname, data=data)
            else:
                ds = fo.create_dataset(vname, data=vars_dict[vname].values) 
        print(('output ->', ofile))
else:
    print('output -> None')




OSError: Unable to synchronously create file (unable to truncate a file which is already open)

In [ ]:
atl13_xr = xr.open_dataset('tmp_subset.h5') 
atl13_xr


<xarray.Dataset> Size: 121kB
Dimensions:                        (phony_dim_0: 313, phony_dim_1: 4)
Dimensions without coordinates: phony_dim_0, phony_dim_1
Data variables: (12/73)
    atl13refid                     (phony_dim_0) float64 3kB ...
    bottom_lat                     (phony_dim_0) float64 3kB ...
    bottom_lon                     (phony_dim_0) float64 3kB ...
    cloud_flag_asr_atl09           (phony_dim_0) float32 1kB ...
    cloud_flag_atm_atl09           (phony_dim_0) float32 1kB ...
    cycle_number                   (phony_dim_0) int8 313B ...
    ...                             ...
    sseg_start_lon                 (phony_dim_0) float64 3kB ...
    stdev_water_surf               (phony_dim_0) float32 1kB ...
    subsurface_attenuation         (phony_dim_0) float32 1kB ...
    subsurface_backscat_ampltd     (phony_dim_0) float32 1kB ...
    transect_id                    (phony_dim_0) float32 1kB ...
    water_depth                    (phony_dim_0) float32 1kB ...